In [2]:
import numpy as np
import pandas as pd
import math
import time
from sklearn.model_selection import train_test_split

In [137]:
rdata = pd.read_csv('sample_treino.csv') #read the data

data = rdata.iloc[:,0:len(rdata.columns)-1] # remove last
data = np.c_[np.ones(len(rdata)),data]      #add extra columns with ones

Y = rdata.iloc[:,len(rdata.columns)-1:len(rdata.columns)].values #saving the Y the cra
w = np.zeros([len(rdata.columns),1]) #the variables

In [16]:
data_train, data_test, Y_train, Y_test = train_test_split(data, Y, test_size=0.2, random_state=0)

Error function

$ MSE(w) = \frac{1}{N} \sum_{i=1}^{N} {(X^T \cdot W - Y_i)^2}$

In [140]:
def MSE(X,Y,w):
    return np.sum(np.power(((X @ w)-Y),2))/len(X)

In [141]:
MSE(data_train,Y_train,w)

55.13513034200704

$ w^{t+1} = w^t -2H^t (y-H_w)$

$\nabla RSS(w) = \nabla [(y - Hw)^T (y - Hw)]$

$\nabla RSS(w) = -2H^T (y - Hw)$

In [34]:
def grad_rss(w,X,Y):
    return X.T @ ( (X @ w.flatten()) - Y.flatten() )/len(X)

In [142]:
def nse(X,Y,w):
    return ((X @ w.T)-Y).flatten()

#def ng(w,X,Y):
#    return np.dot((-2 * X.T),(Y - (X @ w.T)))

def ng(w,X,Y):
    return -2 * np.dot((X.T),(Y - (np.dot(X, w))))

#print(data_train @ w.flatten())
#print(Y_train)
#print(data_train @ w.T)

#print(Y - data_train @ w.T)
#print((-2 * data_train.T) @ (Y - data_train @ w.flatten()))

#print(np.sum(nse(data_train,Y_train,w)))
#print(grad_rss(w,data_train,Y_train))
print(ng(w,data_train,Y_train))

[[-1033.0308985 ]
 [-7397.31511437]
 [-8725.3806738 ]
 [-7779.99871014]
 [-8502.9771935 ]
 [-6622.72001657]]


$ w^{t+1} = w^t + \gamma * \nabla RSS(w) $

In [143]:
def step_gradient(w,X,Y,learningRate):
    #return w - ((learningRate/len(X)) * np.sum(X * (X @ w.T - Y), axis=0) )
    #return w - ((learningRate) * grad_rss(w,X,Y) )
    return w - (learningRate * ng(w,X,Y))

In [144]:
def gradient_descent_runner(starting_w,X,Y, learning_rate, epsilon):
    w = starting_w
    grad = epsilon + 1
    i = 0
    while (np.linalg.norm(grad)>=epsilon):
        w = step_gradient(w, X, Y, learning_rate)
        #print('{:.3f}'.format(np.linalg.norm(grad)),np.linalg.norm(grad)>=epsilon)
        grad = ng(w,X,Y)
        if (np.linalg.norm(grad) == np.inf):
            return w,i
            
        i+= 1
    return w,i

sss

In [155]:
tic = time.time()
model,i = gradient_descent_runner(w,data_train,Y_train,0.00003,0.9)
#model,i = gradient_descent_runner(w,data_train,Y_train,0.0000001,0.9)
toc = time.time()

print("Gradiente descendente convergiu em {0} iterações, com  error = {1}".format(i, MSE(data_train,Y_train,model)))
print("f(x) = {0:.3f}xCálculo1 + {1:.3f}xLPT + {2:.3f}xP1 + {3:.3f}xIC + {4:.3f}xCalculo2".format(model[1][0], model[2][0],
                                                                                                  model[3][0], model[4][0],
                                                                                                  model[5][0])
     )
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")

Gradiente descendente convergiu em 27668 iterações, com  error = 0.38922087514927617
1.210366867205658
f(x) = 0.078xCálculo1 + 0.077xLPT + 0.195xP1 + 0.444xIC + -0.019xCalculo2
Versão vetorizada rodou em: 739.4671440124512 ms


sdsds

In [41]:
MSE(data_test,Y_test,model)
print(model[0][0])

0.7249223444895555


## sklearn version

In [65]:
from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()  
tic = time.time()
regressor.fit(data_train[:,1:], Y_train)
toc = time.time()

#print("f(x) = {0:.3f}xCálculo1 + {1:.3f}xLPT + {2:.3f}xP1 + {3:.3f}xIC + {4:.3f}xCalculo2".format(regressor.coef_[0][0],
#                                                                                                  regressor.coef_[0][1],
#                                                                                                  regressor.coef_[0][2],
#                                                                                                  regressor.coef_[0][3],
#                                                                                                  regressor.coef_[0][4])
#     )
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")
regressor.coef_

Versão vetorizada rodou em: 11.99793815612793 ms


array([[ 0.05880505,  0.0398723 ,  0.19673024,  0.39718294, -0.02694486]])

In [67]:
regressor.intercept_

array([2.08412457])

In [66]:
regressor.coef_

array([[ 0.05880505,  0.0398723 ,  0.19673024,  0.39718294, -0.02694486]])

In [52]:
rdata

,Cálculo1,LPT,P1,IC,Cálculo2,cra
0,8.7,10.0,9.0,9.1,8.4,8.477647
1,7.0,7.0,7.7,7.0,6.2,6.851724
2,8.6,9.8,7.9,9.6,8.7,9.090588
3,7.8,8.3,6.8,8.2,8.0,7.283516
4,5.2,9.3,5.0,8.5,5.0,7.205747
5,6.1,9.2,9.1,9.3,5.6,7.808235
6,6.3,9.6,8.3,8.6,6.7,8.858824
7,7.5,8.9,7.5,7.9,5.0,6.158824
8,6.6,9.0,5.0,7.9,5.0,6.730588
9,7.5,6.5,7.3,9.2,8.2,7.079310


array([[ 7.8 ,  7.3 ,  9.1 ,  9.8 ],
       [ 7.2 ,  7.75,  5.7 ,  7.  ],
       [ 7.2 ,  8.5 ,  7.7 ,  8.4 ],
       [ 6.3 ,  9.6 ,  8.3 ,  8.6 ],
       [ 8.7 ,  8.7 ,  8.1 ,  8.9 ],
       [ 7.5 ,  9.5 ,  8.6 ,  8.5 ],
       [ 5.3 ,  7.6 ,  7.4 ,  9.  ],
       [ 5.  ,  7.  ,  8.  ,  7.2 ],
       [ 9.8 ,  7.6 , 10.  , 10.  ],
       [ 6.7 ,  9.3 ,  7.4 ,  8.3 ],
       [ 9.5 ,  8.  ,  6.  ,  7.1 ],
       [ 7.8 ,  8.3 ,  6.8 ,  8.2 ],
       [ 7.2 ,  9.2 ,  7.  ,  8.8 ],
       [ 8.7 ,  8.2 ,  8.1 ,  9.9 ],
       [ 8.3 ,  9.6 ,  7.  ,  9.1 ],
       [ 5.8 , 10.  ,  8.2 ,  7.4 ],
       [ 8.8 ,  8.3 ,  6.6 ,  7.  ],
       [ 7.2 ,  7.6 ,  7.9 ,  7.7 ],
       [ 6.2 ,  7.5 ,  8.1 ,  7.2 ],
       [ 5.9 ,  7.  ,  6.1 ,  7.8 ],
       [ 7.8 ,  9.2 ,  7.6 ,  9.8 ],
       [ 5.2 ,  9.3 ,  5.  ,  8.5 ],
       [ 7.2 ,  9.5 ,  8.9 ,  8.1 ],
       [ 7.  ,  9.6 ,  8.6 ,  9.3 ],
       [ 5.3 ,  9.5 ,  9.1 ,  8.  ],
       [ 7.  ,  9.  ,  5.9 ,  6.  ],
       [ 6.1 ,  9.2 ,  9.1 ,  9.3 ],
 

In [47]:
#coeff_df = pd.DataFrame(regressor.coef_, rdata.columns[:-1], columns=['Coefficient'])  
#coeff_df  
#y_pred = regressor.predict(X_test)
Y_pred = regressor.predict(data_test[:,1:len(rdata.columns)-1])
#df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})  
#df

In [48]:
from sklearn import metrics 
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))

Mean Squared Error: 0.5670409553488172
